# O programa Rodar-Programa-Alguns-Campos
contém alguns campos do Projeto Original das APIs 
que serão tratabalhas conforme cada programa para tratamento
dos dados obtidos:

In [2]:
import requests 
import pandas as pd
import time
from datetime import datetime
import csv

print("Executando codigo...")

# Constant fields to be sure all results rows will be the same
# FIELDS = ['vacina_categoria_codigo',
#          'vacina_lote',
#          'estabelecimento_razaoSocial',
#          'estalecimento_noFantasia',
#          'paciente_endereco_cep',
#          'estabelecimento_uf',
#          'sistema_origem',
#          'paciente_endereco_nmMunicipio',
#          'paciente_racaCor_valor',
#          'paciente_dataNascimento',
#          'vacina_fabricante_nome',
#          'estabelecimento_municipio_nome',
#          'paciente_idade',
#          'vacina_grupoAtendimento_codigo',
#          'vacina_descricao_dose',
#          'paciente_nacionalidade_enumNacionalidade',
#          'vacina_codigo',
#          'estabelecimento_municipio_codigo',
#          'paciente_id',
#          'vacina_categoria_nome',
#          'vacina_dataAplicacao',
#          'document_id',
#          'paciente_enumSexoBiologico',
#          'estabelecimento_valor',
#          'paciente_endereco_uf',
#          'paciente_endereco_coPais',
#          'paciente_endereco_nmPais',
#          'paciente_endereco_coIbgeMunicipio',
#          'vacina_grupoAtendimento_nome',
#          'paciente_racaCor_codigo',
#          'vacina_fabricante_referencia']


FIELDS = [
    
           'vacina_fabricante_nome',
            'paciente_idade',
            'paciente_id',
            'vacina_dataAplicacao',
            'paciente_endereco_uf',
            'vacina_descricao_dose']
    


# Function to only get the fields from the hits
def get_values(hits, fields):
    return [[hit[field]  if field in hit.keys() 
                         else "" for field in fields] for hit in hits]

# Function to retry request in case of timeout or error
def get_request(url, user, password, body, retries=3):
    for i in range(retries):
        response = requests.post(url, auth=(user, password), json=request_body).json()
        if "error" not in response.keys():
            return response
        else:
            print("Retrying request")
    return None

saving_path = f'vacinas_{datetime.now().strftime("%Y-%m-%d-%H%M")}.csv'

with open(saving_path, "w") as csv_file: 
    writer = csv.writer(csv_file, delimiter=';')

    # Add column names to csv file
    writer.writerow(FIELDS)

    ## Start first request
    url = "https://imunizacao-es.saude.gov.br/_search?scroll=1m"
    request_body = {"size": 10000}
    user = "imunizacao_public"
    password = "qlto5t&7r_@+#Tlstigi"
    response = requests.post(url, auth=(user, password), json=request_body).json()

    hits = response.get("hits").get("hits")
    hits = [hit.get("_source") for hit in hits]
    
    # Write values to CSV file
    writer.writerows(get_values(hits, FIELDS))
    scroll_id = response.get("_scroll_id")
    counts = len(hits)
    while True:
        request_body = {"scroll_id": scroll_id, "scroll": "1m"}
        url = "https://imunizacao-es.saude.gov.br/_search/scroll"
        t0 = time.time()
        response = get_request(url, user, password, request_body)
        t1 = time.time()
        request_time = t1 - t0
        if response:        
            scroll_id = response.get("_scroll_id")
            hits = response.get("hits").get("hits")
            hits = [hit.get("_source") for hit in hits]
            if len(hits)==0:
                break
            t0 = time.time()
            # Write values to CSV file
            writer.writerows(get_values(hits, FIELDS))
            t1 = time.time()
            append_time = t1-t0
            counts = counts+len(hits)
            # print(f"hits len: {len(hits)}, total results: {counts}, 
            #      request time: {request_time:.2f}, append time: {append_time:.2f}")
                 
    
    
            print(f"hits len: {len(hits)}, total results: {counts}")
        else:
            print("Request error")
            break
print("Done")




Executando codigo...
hits len: 10000, total results: 20000
hits len: 10000, total results: 30000
hits len: 10000, total results: 40000
hits len: 10000, total results: 50000
hits len: 10000, total results: 60000
hits len: 10000, total results: 70000
hits len: 10000, total results: 80000
hits len: 10000, total results: 90000
hits len: 10000, total results: 100000
hits len: 10000, total results: 110000
hits len: 10000, total results: 120000
hits len: 10000, total results: 130000
hits len: 10000, total results: 140000
hits len: 10000, total results: 150000
hits len: 10000, total results: 160000
hits len: 10000, total results: 170000
hits len: 10000, total results: 180000
hits len: 10000, total results: 190000
hits len: 10000, total results: 200000
hits len: 10000, total results: 210000
hits len: 10000, total results: 220000
hits len: 10000, total results: 230000
hits len: 10000, total results: 240000
hits len: 10000, total results: 250000
hits len: 10000, total results: 260000
hits len: 10

In [3]:

# Calculando a porcentagem dos vacinados/população brasileira:

print(counts)

porcentagem = (counts/211795632)


print('Porcentagem Brasileiros Vacinados:', porcentagem)
      
      



44546756
Porcentagem Brasileiros Vacinados: 0.21032896466911083


In [ ]:
# Após a geração do arquivo: Vacinas.csv --> amostragem de 5, 10 ou 20%